In [ ]:
#mounting Google Drive to save model
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%capture
# !pip install scispacy
# !pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.4/en_core_sci_sm-0.5.4.tar.gz
# !pip install --upgrade scispacy
!pip install -q torch_geometric
!pip install -q transformers
!pip install -q datasets
# !pip install -q sentencepiece
!pip install rouge
!pip install bert_score
!pip install rouge-score
!pip install sacrebleu
# !pip install --upgrade datasets
!pip install evaluate

In [ ]:
%%capture
# import spacy
# import scispacy
# from scispacy.linking import EntityLinker

from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
import torch


import tensorflow as tf
import numpy as np
import pandas as pd
import torch
import transformers
import os
import re
import json
import bert_score
import rouge
import sacrebleu
import evaluate

import matplotlib.pyplot as plt
import seaborn as sns

from datasets import load_dataset
from datasets import DatasetDict


# from transformers import pipeline, T5Tokenizer, TFT5Model, T5ForConditionalGeneration, AutoTokenizer
from transformers import Trainer, TrainingArguments
from transformers import LongformerTokenizerFast, BartForConditionalGeneration, Trainer, TrainingArguments, BartTokenizer
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoModelForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq

#evaluation packages
#rogue score
from rouge import Rouge
from evaluate import load
# BERTScore leverages the pre-trained contextual embeddings from BERT and matches words in candidate and reference sentences by cosine similarity.
from bert_score import BERTScorer

import shutil

In [ ]:
ds = load_dataset("Bilal-Mamji/Medical-summary")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/567 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.9M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.20M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/615k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9250 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/250 [00:00<?, ? examples/s]

In [ ]:
#validating dataset
print(f"{len(ds['train'])} training pairs")
print(f"{len(ds['validation'])} validation pairs")
print(f"{len(ds['test'])} test pairs")

9250 training pairs
500 validation pairs
250 test pairs


In [ ]:
#renaming headers and deleting instructions column
ds = ds.remove_columns(['instruction']) #not relevant to the model baseline
ds = ds.rename_column('input', 'input_text')
ds = ds.rename_column('output', 'target_text')


In [ ]:
#saving data to a dataset DatasetDict to work with DistilBART tokenization
dataset = DatasetDict({
    'train': ds['train'],
    'validation': ds['validation'],
    'test': ds['test']
})

In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer
model = BartForConditionalGeneration.from_pretrained("sshleifer/distilbart-cnn-12-6")
tokenizer = BartTokenizer.from_pretrained("sshleifer/distilbart-cnn-12-6")

config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

##2. Graph Convoluted Network (GNC) Setup

In [ ]:
def sliding_window_chunking(text, tokenizer, max_length=900, stride=256):
    """
    Split text into overlapping chunks using a sliding window.
    """
    tokens = tokenizer(text, truncation=False, return_tensors="pt")["input_ids"][0]
    chunks = []
    start = 0
    while start < len(tokens):
        end = min(start + max_length, len(tokens))
        chunk = tokens[start:end]
        chunk_text = tokenizer.decode(chunk, skip_special_tokens=True)
        chunks.append(chunk_text)
        start += max_length - stride
    return chunks

##3. Preprocessing Function

##4. KeyBERT Approach

In [ ]:
%%capture
!pip install keybert
!pip install transformers datasets keybert

In [ ]:
from keybert import KeyBERT
from transformers import BartTokenizer

#initialize KeyBERT
kw_model = KeyBERT(model="all-MiniLM-L6-v2")

# KeyBERT preprocessing function
def preprocess_with_keybert(batch):
    key_phrases = kw_model.extract_keywords(
        batch["input_text"],
        keyphrase_ngram_range=(1, 3),  # Single words, bigrams, trigrams
        stop_words="english",          # General stop word removal
        top_n=15,                      # Extract top 15 key phrases
        use_maxsum=True,               # Reduce redundancy
        nr_candidates=20,              # Number of candidates for max-sum selection
        use_mmr=True,                  # Relevance-diversity trade-off
        diversity=0.7                  # Adjust diversity of phrases
    )
    #combine key phrases into a single string
    batch["input_text"] = " ".join([phrase[0] for phrase in key_phrases])
    return batch

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# from keybert import KeyBERT
# from transformers import BartTokenizer

# # Initialize KeyBERT
# kw_model = KeyBERT(model="all-MiniLM-L6-v2")

# # KeyBERT preprocessing function
# def preprocess_with_keybert(batch):
#     # Process each input text in the batch individually
#     processed_texts = []
#     for text in batch["input_text"]:
#         key_phrases = kw_model.extract_keywords(
#             text,
#             keyphrase_ngram_range=(1, 3),  # Single words, bigrams, trigrams
#             stop_words="english",          # General stop word removal
#             top_n=15,                      # Extract top 15 key phrases
#             use_maxsum=False,               # Reduce redundancy
#             nr_candidates=10,              # Number of candidates for max-sum selection
#             use_mmr=False,                  # Relevance-diversity trade-off
#             diversity=0.7                  # Adjust diversity of phrases
#         )
#         # Combine key phrases into a single string for each input text
#         processed_texts.append(" ".join([phrase[0] for phrase in key_phrases if isinstance(phrase, tuple)]))

#     # Update the batch with the processed texts
#     batch["input_text"] = processed_texts
#     return batch

In [ ]:
#aapply KeyBERT to the dataset
processed_dataset = dataset.map(preprocess_with_keybert, batched=True, batch_size=128)

Map:   0%|          | 0/9250 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

In [ ]:
processed_dataset.save_to_disk("/content/drive/My Drive/KeyBERT_DistilBART_Processed_Dataset")

Saving the dataset (0/1 shards):   0%|          | 0/9250 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/500 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/250 [00:00<?, ? examples/s]

In [ ]:
from transformers import BartTokenizer


tokenizer = BartTokenizer.from_pretrained("sshleifer/distilbart-cnn-12-6")


def tokenize_data(batch):
    inputs = tokenizer(batch["input_text"], max_length=900, truncation=True, padding="max_length")
    targets = tokenizer(batch["target_text"], max_length=600, truncation=True, padding="max_length")
    inputs["labels"] = targets["input_ids"]
    return inputs

#tokenize dataset
tokenized_dataset = processed_dataset.map(tokenize_data, batched=True, remove_columns=["input_text", "target_text"])

Map:   0%|          | 0/9250 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset.save_to_disk("/content/drive/My Drive/KeyBERT_DistilBART_Tokenized_Dataset")

Saving the dataset (0/1 shards):   0%|          | 0/9250 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/500 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/250 [00:00<?, ? examples/s]

In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, EarlyStoppingCallback

training_args = Seq2SeqTrainingArguments(
    output_dir="./results_distilbart_keybert",  #directory to save checkpoints
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,             #accumulate gradients for larger effective batch size
    eval_steps=500,
    save_steps=500,
    save_total_limit=2,
    learning_rate=5e-5,
    predict_with_generate=True,
    logging_dir="./logs_distilbart_keybert",
    logging_steps=100,
    eval_strategy="steps",
    load_best_model_at_end=True,               #load the best checkpoint
    metric_for_best_model="eval_loss",         #monitor validation loss
    greater_is_better=False,                   #lower loss is better
    fp16=True
)


early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=3  # Stop training if no improvement after 3 evaluations
)

In [ ]:
# Initialize Trainer
%%time
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    callbacks=[early_stopping_callback]
)


trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss
500,1.530700,1.416006
1000,1.370500,1.311650
1500,1.256700,1.260914
2000,1.244700,1.227593
2500,1.153000,1.209418
3000,1.152200,1.193968


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


CPU times: user 13min 2s, sys: 31.8 s, total: 13min 34s
Wall time: 14min 9s


TrainOutput(global_step=3468, training_loss=1.3459481632008272, metrics={'train_runtime': 847.3172, 'train_samples_per_second': 32.75, 'train_steps_per_second': 4.093, 'total_flos': 3.7739426881536e+16, 'train_loss': 1.3459481632008272, 'epoch': 2.998702983138781})

In [ ]:
# Evaluate the model on the validation set
results = trainer.evaluate(eval_dataset=tokenized_dataset["validation"])
print("Evaluation Results:", results)

Evaluation Results: {'eval_loss': 1.193968415260315, 'eval_runtime': 3.9918, 'eval_samples_per_second': 125.256, 'eval_steps_per_second': 15.782, 'epoch': 2.998702983138781}


In [ ]:

import shutil

# Save the model and tokenizer locally
model.save_pretrained("./distilbart_keybert", safe_serialization=True)
tokenizer.save_pretrained("./distilbart_keybert")

# Path where the model is saved locally
local_model_path = "/content/distilbart_keybert"  # Corrected this path to match the save_pretrained directory

# Path in Google Drive where you want to save the model
drive_model_path = "/content/drive/My Drive/DistilBARTFolder_keybert"

# Copy the entire directory to Google Drive
shutil.copytree(local_model_path, drive_model_path)

print("Model directory uploaded to Google Drive!")

Model directory uploaded to Google Drive!


##4. Evaluations

In [ ]:
#load model from drive
from transformers import BartForConditionalGeneration, BartTokenizer
model = BartForConditionalGeneration.from_pretrained("/content/drive/My Drive/DistilBARTFolder_keybert")
tokenizer = BartTokenizer.from_pretrained("/content/drive/My Drive/DistilBARTFolder_keybert")


/usr/local/lib/python3.10/dist-packages/transformers/models/bart/configuration_bart.py:176: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(


In [ ]:
def generate_prediction(input_text):
    '''function to generate predictions from input_text, this case text dataset.
    This will be used for human evaluation'''
    inputs = tokenizer(input_text, return_tensors="pt", max_length=900, truncation=True, padding=True)

    # Move input tensors to the same device as the model
    input_ids = inputs["input_ids"].to(model.device)

    outputs = model.generate(
        input_ids,
        max_length=512,  #your expected max output length
        do_sample=True,
        top_k=50,        #top-k sampling for diversity
        top_p=0.95,      # nucleus sampling
        temperature=1.0,
        num_beams=4,
        length_penalty=2.0,
        early_stopping=True
    )
    prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return prediction

In [ ]:
#measure entire cell process time
%%time
import random


# random.seed(42)

num_samples = 10
# num_samples = 5

# random_samples = dataset["test"].shuffle(seed=42).select(range(num_samples))
subset_samples = dataset["test"].select(range(num_samples))
# subset_samples = dataset["validation"].select(range(num_samples)) #one time pull
#empty list to save outputs, will be used to export to Google Drive below
results = []

#generate predictions
for idx, row in enumerate(subset_samples):
    input_text = row["input_text"]
    target_text = row["target_text"]
    prediction = generate_prediction(input_text)

    # print(f"\nSample {idx + 1}:")
    # print(f"Input Text:\n{input_text}\n")
    # print(f"Target Text (Ground Truth):\n{target_text}\n")
    # print(f"Model Prediction:\n{prediction}\n")
    # print("-" * 80)


    results.append({
        "Input Text": input_text,
        "Target Text": target_text,
        "Model Prediction": prediction
    })


    print(f"Processed Sample {idx + 1}/{len(subset_samples)}")


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1493: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed in v5. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


Processed Sample 1/5
Processed Sample 2/5
Processed Sample 3/5
Processed Sample 4/5
Processed Sample 5/5
CPU times: user 5min 46s, sys: 2.47 s, total: 5min 49s
Wall time: 1min


##4.1 Human Evaluation

In [ ]:
# Convert results to a DataFrame
df = pd.DataFrame(results)

# Define the output path in Google Drive
output_path = "/content/drive/My Drive/DistilBART_KeyBERT_HE.csv"
# output_path = "/content/drive/My Drive/DistilBART_KeyBERT_validate5.csv"
# Save to CSV
df.to_csv(output_path, index=False)

print(f"Results saved to {output_path}")

Results saved to /content/drive/My Drive/DistilBART_KeyBERT_validate5.csv


##4.2 ROUGE Evaluation

In [ ]:
#load rouge metric
rouge = load("rouge") #lrouge metric using load function
#gpu to local device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#move model to local
model.to(device)

#generate predictions and references
predictions = []
references = []
for row in dataset["test"]:  #iterate through the dataset
    input_text = row["input_text"]
    target_text = row["target_text"]  # extract the target text
    prediction = generate_prediction(input_text)
    predictions.append(prediction)
    references.append(target_text)

rouge_results = rouge.compute(predictions=predictions, references=references)
print("ROUGE Results:", rouge_results)

ROUGE Results: {'rouge1': 0.4728918496917739, 'rouge2': 0.2000540064547909, 'rougeL': 0.2905794092136986, 'rougeLsum': 0.39110374413076543}


##4.3 BLEU Evaluation

In [ ]:

#use generate_prediction function
# predictions, references = generate_predictions(dataset["test"], model, tokenizer, device)
predictions = []
references = []
for row in dataset["test"]:
  prediction = generate_prediction(row['input_text'])
  predictions.append(prediction)
  references.append(row['target_text'])


#BLEU expects references as a list of lists
references = [[ref] for ref in references]


bleu_score = sacrebleu.corpus_bleu(predictions, references)
print("BLEU Score:", bleu_score.score)

BLEU Score: 72.96670055087627


##4.4 BERT Score Evaluation

In [ ]:
#BERT Score function
def evaluate_bertscore(predictions, references, lang="en"):
    #BERTScore
    P, R, F1 = bert_score.score(predictions, references, lang=lang, verbose=True)


    return {
        "BERTScore Precision": P.mean().item(),
        "BERTScore Recall": R.mean().item(),
        "BERTScore F1": F1.mean().item()
    }

print(evaluate_bertscore(predictions, references))

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/8 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/4 [00:00<?, ?it/s]

done in 7.82 seconds, 31.95 sentences/sec
{'BERTScore Precision': 0.8769432902336121, 'BERTScore Recall': 0.866653323173523, 'BERTScore F1': 0.8716947436332703}


In [ ]:
%%capture
!pip install keybert

In [ ]:
from keybert import KeyBERT

kw_model = KeyBERT(model="all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Input text
input_text = "The patient reports chest pain and shortness of breath. They have a history of hypertension and are prescribed Lisinopril."

# Extract keywords
key_phrases = kw_model.extract_keywords(
    input_text,
    keyphrase_ngram_range=(1, 2),  # Single words and bigrams
    stop_words="english",          # Remove common stop words
    top_n=5                        # Extract top 5 key phrases
)

# Print the extracted keywords
print("Key Phrases:", key_phrases)

Key Phrases: [('prescribed lisinopril', 0.6701), ('lisinopril', 0.5904), ('chest pain', 0.5592), ('hypertension prescribed', 0.5264), ('shortness breath', 0.5092)]


In [ ]:
#combine key phrases into a structured input
filtered_input = " ".join([phrase[0] for phrase in key_phrases])
print("Filtered Input:", filtered_input)

Filtered Input: prescribed lisinopril lisinopril chest pain hypertension prescribed shortness breath


In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration
import torch

#load DistilBART model and tokenizer, specifying the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Check for GPU availability
tokenizer = BartTokenizer.from_pretrained("sshleifer/distilbart-cnn-12-6")
model = BartForConditionalGeneration.from_pretrained("sshleifer/distilbart-cnn-12-6").to(device) # Move model to device

#tokenize the filtered input
inputs = tokenizer(filtered_input, return_tensors="pt", max_length=1024, truncation=True)

#move input tensors to the same device as the model
inputs = {k: v.to(device) for k, v in inputs.items()}

#generate a summary
outputs = model.generate(inputs["input_ids"], max_length=50, num_beams=4)
summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Summary:", summary)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1399: UserWarning: Unfeasible length constraints: `min_length` (56) is larger than the maximum possible length (50). Generation will stop at the defined maximum length. You should decrease the minimum length and/or increase the maximum length.
  warnings.warn(


Summary: S: The patient reports experiencing chest pain and shortness of breath for the past 2 days. The patient has a history of hypertension and is currently on lisinopril for chest pain.
O: A chest CT scan revealed a


In [ ]:
from bert_score import score
from sklearn.metrics import accuracy_score
from nltk.translate.bleu_score import corpus_bleu
from rouge_score import rouge_scorer
import re
from collections import defaultdict

def split_soap_text(document):
    """
    Splits a SOAP note document into its four components: Subjective, Objective, Assessment, and Plan.
    """
    patterns = {
        "Subjective": r"(?:Subjective:|S:)(.*?)(?:Objective:|O:|Assessment:|A:|Plan:|P:|$)",
        "Objective": r"(?:Objective:|O:)(.*?)(?:Assessment:|A:|Plan:|P:|$)",
        "Assessment": r"(?:Assessment:|A:)(.*?)(?:Plan:|P:|$)",
        "Plan": r"(?:Plan:|P:)(.*)",
    }

    soap_parts = {}
    for part, pattern in patterns.items():
        match = re.search(pattern, document, re.DOTALL | re.IGNORECASE)
        if match:
            soap_parts[part] = match.group(1).strip()
        else:
            soap_parts[part] = ""
    return soap_parts

def evaluate_bleu_corpus(references, predictions):
    """
    Compute BLEU score for the corpus.
    """
    # Import necessary libraries
    from nltk.translate.bleu_score import corpus_bleu
    import re

    ref_tokens = [[ref.split()] for ref in references]
    pred_tokens = [pred.split() for pred in predictions]
    return corpus_bleu(ref_tokens, pred_tokens)

def evaluate_rouge_corpus(references, predictions):
    """
    Compute average ROUGE scores for the corpus.
    """
    # Import necessary libraries
    from rouge_score import rouge_scorer
    from collections import defaultdict

    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = defaultdict(list)

    for ref, pred in zip(references, predictions):
        scores = scorer.score(ref, pred)
        for key, value in scores.items():
            rouge_scores[key].append(value.fmeasure)

    return {key: sum(values) / len(values) for key, values in rouge_scores.items()}

def evaluate_bert_score_corpus(references, predictions):
    """
    Compute BERTScore for the corpus.
    """
    # Call the bert_score.score function directly
    P, R, F1 = score(predictions, references, lang="en")
    return {
        "BERT_Precision": P.mean().item(),
        "BERT_Recall": R.mean().item(),
        "BERT_F1": F1.mean().item()
    }

# Example lists of predictions and references
predictions = []
references = []
for row in dataset["test"]:
  prediction = generate_prediction(row['input_text'])
  predictions.append(prediction)
  references.append(row['target_text'])

# Import necessary library for regular expressions
import re

# Split SOAP components
all_predictions_split = [split_soap_text(doc) for doc in predictions]
all_references_split = [split_soap_text(doc) for doc in references]

# Evaluate each SOAP component at the corpus level
evaluation_results = {}

for key in ["Subjective", "Objective", "Assessment", "Plan"]:
    pred_parts = [pred[key] for pred in all_predictions_split]
    ref_parts = [ref[key] for ref in all_references_split]

    bleu_score = evaluate_bleu_corpus(ref_parts, pred_parts)
    rouge_scores = evaluate_rouge_corpus(ref_parts, pred_parts)
    bert_scores = evaluate_bert_score_corpus(ref_parts, pred_parts)

    evaluation_results[key] = {
        "BLEU": bleu_score,
        **rouge_scores,
        **bert_scores
    }

# Display results
for key, metrics in evaluation_results.items():
    print(f"{key} Evaluation:")
    for metric, score in metrics.items():
        print(f"  {metric}: {score:.4f}")
    print()

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1493: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed in v5. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Subjective Evaluation:
  BLEU: 0.1543
  rouge1: 0.4811
  rouge2: 0.2740
  rougeL: 0.3772
  BERT_Precision: 0.9048
  BERT_Recall: 0.8890
  BERT_F1: 0.8965

Objective Evaluation:
  BLEU: 0.1059
  rouge1: 0.3644
  rouge2: 0.1733
  rougeL: 0.2539
  BERT_Precision: 0.8612
  BERT_Recall: 0.8577
  BERT_F1: 0.8589

Assessment Evaluation:
  BLEU: 0.1018
  rouge1: 0.3432
  rouge2: 0.1426
  rougeL: 0.2783
  BERT_Precision: 0.8541
  BERT_Recall: 0.8535
  BERT_F1: 0.8535

Plan Evaluation:
  BLEU: 0.0766
  rouge1: 0.4011
  rouge2: 0.1392
  rougeL: 0.2532
  BERT_Precision: 0.8503
  BERT_Recall: 0.8349
  BERT_F1: 0.8423

